In [3]:
import zipfile as zf
ROOT_DIR = "/home/sirui/INNOMAUS"
files = zf.ZipFile("/home/sirui/INNOMAUS/model.zip", 'r')
files.extractall(ROOT_DIR)
files.close()

In [2]:
from PIL import Image
import os
import tifffile
import numpy as np
from scipy import interpolate

from skimage.transform import resize
from scipy.ndimage import zoom

def resize_image(input_path, output_path, target_size):
    print(input_path)
    img = tifffile.imread(input_path)
    imag = img
    img = Image.fromarray(img)
    img = img.resize((img.size[0] // 6, img.size[1] // 6), Image.BILINEAR)
    img = np.array(img)
    tifffile.imsave(output_path, img)
    print(f"Resized {input_path} to {output_path}")
    # except Exception as e:
    #     print(f"Error processing {input_path}: {str(e)}")
def inter(input_path, output_path, target_size):
    array = tifffile.imread(input_path)
    h, w = array.shape
    new_h, new_w = np.floor_divide((h, w), target_size)
    x = np.linspace(0, w - 1, w)
    y = np.linspace(0, h - 1, h)
    new_x = np.linspace(0, w - 1, new_w)
    new_y = np.linspace(0, h - 1, new_h)
    f = interpolate.interp2d(x, y, array, kind='linear')
    array_down = f(new_x, new_y)
    tifffile.imsave(output_path, array_down)
    print(f"Resized {input_path} to {output_path}")
    # array_down = resize(array, (new_h, new_w), order=1, anti_aliasing=True)
    return array_down

def process_directory(input_directory, output_directory, target_size):

    for root, _, files in os.walk(input_directory):
        for file in files:
            if file.lower().endswith(".tif"):
                name = os.path.basename(root)
                print(name)
                input_path = os.path.join(root, file)
                output_path1 = os.path.join(output_directory, name)
                if not os.path.exists(output_path1):
                    os.makedirs(output_path1)
                output_path = os.path.join(output_path1, file)
                print(output_path)
                resize_image(input_path, output_path, target_size)

if __name__ == "__main__":
    input_directory = "/home/sirui/INNOMAUS/b"
    output_directory = "/home/sirui/INNOMAUS/output"
    target_size = 6

    process_directory(input_directory, output_directory, target_size)

65mm
/home/sirui/INNOMAUS/output/65mm/r2d_bln1_swmm_b_65mm_4500V.tif
/home/sirui/INNOMAUS/b/65mm/r2d_bln1_swmm_b_65mm_4500V.tif
Resized /home/sirui/INNOMAUS/b/65mm/r2d_bln1_swmm_b_65mm_4500V.tif to /home/sirui/INNOMAUS/output/65mm/r2d_bln1_swmm_b_65mm_4500V.tif
65mm
/home/sirui/INNOMAUS/output/65mm/r2d_bln1_swmm_b_65mm_3600V.tif
/home/sirui/INNOMAUS/b/65mm/r2d_bln1_swmm_b_65mm_3600V.tif


/tmp/ipykernel_2009962/596589009.py:17: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(output_path, img)


Resized /home/sirui/INNOMAUS/b/65mm/r2d_bln1_swmm_b_65mm_3600V.tif to /home/sirui/INNOMAUS/output/65mm/r2d_bln1_swmm_b_65mm_3600V.tif
65mm
/home/sirui/INNOMAUS/output/65mm/r2d_bln1_swmm_b_65mm_4800H.tif
/home/sirui/INNOMAUS/b/65mm/r2d_bln1_swmm_b_65mm_4800H.tif
Resized /home/sirui/INNOMAUS/b/65mm/r2d_bln1_swmm_b_65mm_4800H.tif to /home/sirui/INNOMAUS/output/65mm/r2d_bln1_swmm_b_65mm_4800H.tif
65mm
/home/sirui/INNOMAUS/output/65mm/r2d_bln1_swmm_b_65mm_3300H.tif
/home/sirui/INNOMAUS/b/65mm/r2d_bln1_swmm_b_65mm_3300H.tif
Resized /home/sirui/INNOMAUS/b/65mm/r2d_bln1_swmm_b_65mm_3300H.tif to /home/sirui/INNOMAUS/output/65mm/r2d_bln1_swmm_b_65mm_3300H.tif
65mm
/home/sirui/INNOMAUS/output/65mm/r2d_bln1_swmm_b_65mm_1800U.tif
/home/sirui/INNOMAUS/b/65mm/r2d_bln1_swmm_b_65mm_1800U.tif
Resized /home/sirui/INNOMAUS/b/65mm/r2d_bln1_swmm_b_65mm_1800U.tif to /home/sirui/INNOMAUS/output/65mm/r2d_bln1_swmm_b_65mm_1800U.tif
65mm
/home/sirui/INNOMAUS/output/65mm/r2d_bln1_swmm_b_65mm_5100U.tif
/home/sirui